In [3]:
import pandas as pd

In [4]:
fz_meat = pd.read_excel('/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Data/Data/Fz Meat_POS_2020.xlsx')
fz_meat2 = pd.read_excel('/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Data/Data/Fz Meat_POS_2021.xlsx')
fz_meat3 = pd.read_excel('/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Data/Data/Fz Meat_POS_2022.xlsx')
fz_meat4 = pd.read_excel('/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Data/Data/Fz Meat_POS_2023.xlsx')
fz_meat5 = pd.read_excel('/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Data/Data/Fz Meat_POS_2024.xlsx')
product_attribute = pd.read_excel('/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Cleaned Data/Product Attributes.xlsx')


In [5]:
#fz_meat[fz_meat['UPC 13 digit'] == 624967436101]

In [6]:
#fz_meat.head()

In [7]:
#fz_meat['Geography'].unique()

In [8]:
# mapping location column


In [9]:
#test_data = fz_meat['Time'][0]
#sep_data = test_data.split('-')
#print(sep_data)

In [10]:
# Imputation
def imputation_process(row):
    row['Incremental Units'] = row['Unit Sales'] - row['Base Unit Sales']
    row['Incremental Volume'] = row['Volume Sales'] - row['Base Volume Sales']
    row['Incremental Dollars'] = row['Dollar Sales'] - row['Base Dollar Sales']
    return row

# Split Geography column and only save its location
def extract_location(col):
    return col.split(' - ')[0]

# convert time into year month and day
def parse_time(col):
    # Year, month, day
    sep_data = col.split('-')
    year = int('20' + sep_data[2])
    month = int(sep_data[0][-2:])
    day = int(sep_data[1])
    # Quarter of given date
    quarter = (month - 1) // 3 + 1
    return pd.Series([year, quarter, month], index = ['Year', 'Quarter', 'Month'])

# Split product column and only save company name and product name
def parse_product(col):  
    return col.split(' - ')[0]


# Based on length of company name, select product name
def extract_product_name(full_string, brand_name):
    brand_name_length = len(brand_name)
    words = full_string[brand_name_length:].strip()
    return words

# # Arrange columns order
def change_column_order(df):
    final_order = [
        'Geography', 'Year', 'Quarter', 'Month', 'Season', 'Manufacturer Name',
        'Brand Name', 'Product Name', 'Flavor / Scent', 'Meat Source',
        'Unit Sales', 'Volume Sales', 'Dollar Sales', 'Price per Unit', 
        'Price per Volume', 'Base Unit Sales', 'Base Volume Sales', 'Base Dollar Sales',
        'Incremental Units', 'Incremental Volume', 'Incremental Dollars', 
        'ACV Weighted Distribution'
    ]
    return df[final_order]

def merge_content(df1, df2):
    combined = df1.merge(df2, how = 'left', on = 'Product')
    return combined

def map_season(month):
    # Northern Hemisphere standard mapping
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'


# Full tranform process
def transform_data(df, df2):
    table1 = merge_content(df, df2)
    # Used customized function
    table1['Geography'] = table1['Geography'].map(extract_location)
    table1[['Year', 'Quarter', 'Month']] = table1['Time'].apply(parse_time)
    table1['Product'] = table1['Product'].map(parse_product)
    table1['Product Name'] = table1.apply(lambda row: extract_product_name(row['Product'], row['Brand Name']), axis=1)
    #df['Product Name'] = df.apply(extract_product_name(df['Product'], df['Brand']), axis = 1)
    
    # Mapping quarter to seasonality
    table1['Season'] = table1['Month'].apply(map_season)
    # Change column order
    transformed = change_column_order(table1).apply(imputation_process, axis = 1)
    return transformed

In [11]:
# Transform data from 2020 to 2024
fz_meat = transform_data(fz_meat, product_attribute)
fz_meat2 = transform_data(fz_meat2, product_attribute)
fz_meat3 = transform_data(fz_meat3, product_attribute)
fz_meat4 = transform_data(fz_meat4, product_attribute)
fz_meat5 = transform_data(fz_meat5, product_attribute)

In [12]:
#fz_meat.head()

In [13]:
# group = fz_meat.groupby(['Geography', 'Year','Brand']).agg({'Unit Sales':'mean', 'Base Unit Sales': 'mean'})
# group['Diff'] = group['Unit Sales'] - group['Base Unit Sales']
# group

In [14]:
# Combined & export
full_fz_meat = pd.concat([fz_meat, fz_meat2, fz_meat3, fz_meat4, fz_meat5])
full_fz_meat.count()
#output_file = '/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/fz_meat_2020.csv'
#full_fz_meat.to_csv(output_file, index=False)

Geography                    1194776
Year                         1194776
Quarter                      1194776
Month                        1194776
Season                       1194776
Manufacturer Name            1194776
Brand Name                   1194776
Product Name                 1194776
Flavor / Scent               1194055
Meat Source                   309217
Unit Sales                   1194776
Volume Sales                 1194776
Dollar Sales                 1194776
Price per Unit               1194776
Price per Volume             1194776
Base Unit Sales              1194776
Base Volume Sales            1194776
Base Dollar Sales            1194776
Incremental Units            1194776
Incremental Volume           1194776
Incremental Dollars          1194776
ACV Weighted Distribution    1194776
dtype: int64

In [16]:
full_fz_meat[full_fz_meat['Season'] == 'Summer']

,Geography,Year,Quarter,Month,Season,Manufacturer Name,Brand Name,Product Name,Flavor / Scent,Meat Source,...,Dollar Sales,Price per Unit,Price per Volume,Base Unit Sales,Base Volume Sales,Base Dollar Sales,Incremental Units,Incremental Volume,Incremental Dollars,ACV Weighted Distribution
30070,Total US,2020,2,6,Summer,AL SAFA HALAL,AL SAFA HALAL,FROZEN BEEF HOTDOG 12.6 OZ,REGULAR,BEEF,...,1302.075928,6.467177,8.212288,201.336053,158.552142,1302.075928,0.000000,0.000000,0.000000,0.070269
30071,Total US,2020,2,6,Summer,AL SAFA HALAL,AL SAFA HALAL,FROZEN CHICKEN FRANKFURTER 12 OZ,REGULAR,CHICKEN,...,668.474444,3.964708,5.286278,168.606216,126.454662,668.474444,0.000000,0.000000,0.000000,0.087408
30072,Total US,2020,2,6,Summer,COUNTRY FRESH MEATS INC,COUNTRY FRESH MEATS,FROZEN PORK AND BEEF HOTDOG 16 OZ,COLBY CHEESE,PORK AND BEEF,...,575.641133,5.990000,5.990000,96.100356,96.100356,575.641133,0.000000,0.000000,0.000000,0.003759
30073,Total US,2020,2,6,Summer,FAIR MARKET INC,NOBRAND,FROZEN BEEF HOTDOG 80 OZ,CHEDDAR JALAPENO,BEEF,...,779.787416,7.859010,1.571802,99.222087,496.110435,779.787416,0.000000,0.000000,0.000000,0.067965
30074,Total US,2020,2,6,Summer,MARTIN PUREFOODS CORP,MARTIN PUREFOODS,FROZEN BEEF AND PORK HOTDOG 12 OZ,REGULAR,BEEF AND PORK,...,58.503791,5.924244,7.898992,9.875318,7.406489,58.503791,0.000000,0.000000,0.000000,0.011032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265380,West,2023,3,8,Summer,HILLSHIRE BRANDS CO,JIMMY DEAN,FROZEN TURKEY SAUSAGE REGULAR 18.3 OZ,REGULAR,TURKEY,...,1214.224761,8.311765,7.266799,146.085068,167.092101,1214.224761,0.000000,0.000000,0.000000,0.633413
265381,West,2023,3,8,Summer,ULIS FAMOUS SAUSAGE,ULIS FAMOUS SAUSAGE,FROZEN CHICKEN SAUSAGE ROSEMARY 16 OZ,ROSEMARY,CHICKEN,...,63.315116,9.490000,9.490000,6.671772,6.671772,63.315116,0.000000,0.000000,0.000000,0.026833
265382,West,2023,3,8,Summer,ULIS FAMOUS SAUSAGE,ULIS FAMOUS SAUSAGE,FROZEN LAMB SAUSAGE REGULAR 16 OZ,REGULAR,LAMB,...,122.026710,18.290000,18.290000,6.671772,6.671772,122.026710,0.000000,0.000000,0.000000,0.026833
265383,West,2023,3,8,Summer,ULIS FAMOUS SAUSAGE,ULIS FAMOUS SAUSAGE,FROZEN PORK BRATWURST REGULAR 16 OZ,REGULAR,PORK,...,721.443886,9.723816,9.723816,74.193496,74.193496,721.443886,0.000000,0.000000,0.000000,0.299366
